In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
import imageio.v3 as imageio
from matplotlib import pyplot as plt
from google.colab import drive

In [ ]:
# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = os.getcwd() + '/drive/My Drive/My Final Year Project Folder 2023/'

In [ ]:
dataset_voxel = np.load(path + "voxel_grids.npy")

In [ ]:
model =  tf.keras.models.load_model(path + 'my_unet2.h5')

In [ ]:
rotation_matrix = np.asarray([
    [ 9.99964757e-01, -2.59721003e-04, -8.39154021e-03],
     [3.31785339e-04,  9.99963072e-01,  8.58748592e-03],
      [ 8.38899998e-03, -8.58996746e-03,  9.99927916e-01]
      ])

In [ ]:
gc_R = np.broadcast_to(array=rotation_matrix[2], shape=[1, 3])

In [ ]:
output = model(tf.keras.utils.normalize(x=tf.expand_dims(dataset_voxel[0][:176, :336, :], axis=0)))

In [ ]:
hold_struct_param = np.zeros_like(a=output)

In [ ]:
for i in range(hold_struct_param.shape[1]):
    for j in range(hold_struct_param.shape[2]):
        x_vector = np.broadcast_to(array=np.asarray([j, i, 1]), shape=[1, 3])
        hold_struct_param[0, i, j, 0] = output.numpy()[0, i, j, 0] - np.matmul(gc_R , np.transpose(a=x_vector))[0][0]

In [ ]:
predicted_depth = np.abs(1.52/hold_struct_param[0][:176, :336, 0])

In [ ]:
norm = predicted_depth/np.amax(predicted_depth)

In [ ]:
# Get the shape of the pixel array
height, width = height_map.shape

# Create pixel locations as 1D arrays
x_pixels = np.arange(width)
y_pixels = np.arange(height)

# Create a meshgrid of pixel locations
X, Y = np.meshgrid(x_pixels, y_pixels)

# Flatten the pixel values and pixel locations
pixel_values = height_map.flatten()
x_locations = X.flatten()
y_locations = Y.flatten()

# Create a scatter plot
plt.scatter(x_locations, y_locations, c=pixel_values, cmap='viridis')

# Customize plot settings if needed
plt.colorbar(label='Depth')  # Add colorbar with label
plt.title('Depth Map')
plt.xlabel('X Pixels')
plt.ylabel('Y Pixels')

# Show the plot
plt.show()


In [ ]:
# Get the shape of the pixel array
height, width = predicted_depth.shape

# Create pixel locations as 1D arrays
x_pixels = np.arange(width)
y_pixels = np.arange(height)

# Create a meshgrid of pixel locations
X, Y = np.meshgrid(x_pixels, y_pixels)

# Flatten the pixel values and pixel locations
pixel_values = predicted_depth.flatten()
x_locations = X.flatten()
y_locations = Y.flatten()

# Filter pixel values based on a condition
filtered_indices = pixel_values <= 152.43951
filtered_pixel_values = pixel_values[filtered_indices]
filtered_x_locations = x_locations[filtered_indices]
filtered_y_locations = y_locations[filtered_indices]

# Create a scatter plot for filtered pixel values
plt.scatter(filtered_x_locations, filtered_y_locations, c=filtered_pixel_values, cmap='viridis')

# Customize plot settings if needed
plt.colorbar(label='Pixel Value')  # Add colorbar with label
plt.title('Pixel Values < 152.43951')
plt.xlabel('X Pixels')
plt.ylabel('Y Pixels')

# Show the plot
plt.show()

In [ ]:
ground_truth = np.load(path + "depth_image_raw.npy")

In [ ]:
# # This code snippet was found here and modified to work here: https://github.com/nianticlabs/monodepth2
# Without filtering outliers
mask = np.isnan(ground_truth)
abs_rel = np.mean(np.abs(ground_truth - predicted_depth)[~mask] / ground_truth[~mask])

In [ ]:
arr = predicted_depth

In [ ]:
arr[arr > 152.43951] = np.nan

In [ ]:
# This code snippet was found here and modified to work here: https://github.com/nianticlabs/monodepth2
#when outliers are filtered as nans
ground_truth_mask = np.isnan(ground_truth)
predicted_depth_mask = np.isnan(arr)

mask = np.logical_or(ground_truth_mask, predicted_depth_mask)
abs_rel = np.mean(np.abs(ground_truth - arr)[~mask] / ground_truth[~mask])

In [ ]:
# This code snippet was found here and modified to work here: https://github.com/nianticlabs/monodepth2
# when outliers are not filtered
mask = np.isnan(ground_truth)
sq_rel = np.mean(((ground_truth - predicted_depth) ** 2)[~mask] / ground_truth[~mask])

In [ ]:
# This code snippet was found here and modified to work here: https://github.com/nianticlabs/monodepth2
# when outliers are filtered
ground_truth_mask = np.isnan(ground_truth)
predicted_depth_mask = np.isnan(predicted_depth)

mask = np.logical_or(ground_truth_mask, predicted_depth_mask)
sq_rel = np.mean(((ground_truth - predicted_depth) ** 2)[~mask] / ground_truth[~mask])

In [ ]:
# This code snippet was found here and modified to work here: https://github.com/nianticlabs/monodepth2
# with or without filtered outliers
mask = np.isnan(ground_truth) | np.isnan(predicted_depth)
rmse = np.sqrt(((ground_truth - predicted_depth) ** 2)[~mask].mean())

In [ ]:
# This code snippet was found here and modified to work here: https://github.com/nianticlabs/monodepth2
# with or without filtered outliers
mask = np.isnan(ground_truth) | np.isnan(predicted_depth)
rmse_log = np.sqrt(((np.log(ground_truth) - np.log(predicted_depth)) ** 2)[~mask].mean())